In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-06-20 02:57:42--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  62.4MB/s    in 0.4s    

2025-06-20 02:57:43 (62.4 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# Preprocessing: Filter users and books
user_ratings_count = df_ratings['user'].value_counts()
book_ratings_count = df_ratings['isbn'].value_counts()

df_ratings_filtered = df_ratings[
    df_ratings['user'].isin(user_ratings_count[user_ratings_count >= 200].index) &
    df_ratings['isbn'].isin(book_ratings_count[book_ratings_count >= 100].index)
]

# Merge with book data
df_merged = df_ratings_filtered.merge(df_books, on='isbn')

# Create pivot table
df_pivot = df_merged.pivot(index='isbn', columns='user', values='rating').fillna(0)

# Filter books to those in the pivot table
df_books_filtered = df_books[df_books['isbn'].isin(df_pivot.index)]

# Create title to ISBN mapping
title_to_isbn = {}
for title, group in df_books_filtered.groupby('title'):
    title_to_isbn[title] = group['isbn'].iloc[0]

# Create ISBN to title mapping
isbn_to_title = df_books_filtered.set_index('isbn')['title'].to_dict()

# Build the sparse matrix and KNN model
book_matrix = csr_matrix(df_pivot.values)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_matrix)

# Recommendation function
def get_recommends(book = ""):
    if book not in title_to_isbn:
        return [book, []]

    isbn = title_to_isbn[book]
    if isbn not in df_pivot.index:
        return [book, []]

    idx = df_pivot.index.get_loc(isbn)
    distances, indices = model_knn.kneighbors(book_matrix[idx], n_neighbors=6)

    distances = distances.flatten()[1:]
    indices = indices.flatten()[1:]

    distances = distances[::-1]
    indices = indices[::-1]

    recommended_books = []
    for i in range(len(indices)):
        book_isbn = df_pivot.index[indices[i]]
        title = isbn_to_title[book_isbn]
        dist = distances[i]
        recommended_books.append([title, dist])

    return [book, recommended_books]

In [5]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
